## Evaluation_Candidates 

Returns recall and precision _to terminal_

In [1]:
# -*- coding: utf-8 -*-
#!/usr/bin/env python
from __future__ import division
import csv
import sys
import glob
# import spacy
import pandas as pd
from pandas import ExcelFile

In [2]:
# Load english words
# nlp = spacy.load("en_core_web_sm")

# Avoid ascii error
reload(sys)
sys.setdefaultencoding('utf8')

In [3]:
def cleanup_poly(pl):
    new_pl = pl.strip().strip(',').strip('.')
    if len(new_pl)>=2 and new_pl[0] == '(' and new_pl[len(new_pl)-1]==')':
        new_pl=new_pl.rstrip(')').lstrip('(')
    return new_pl

In [4]:
def read_items(ifile):
    items = []
    f = open(ifile)
    lines = f.readlines()
    for line in lines:
        poly = cleanup_poly(line.strip("\n"))
        upoly = u'%s' % (poly)
        items.append(upoly)
    return items

In [5]:
# separate candidates combined by /, :, or –

def split (jfile):
    
    candidate_polymers = read_items(jfile)
    
    candidate_polymers_2 = []
    for string in candidate_polymers:
        if ":" in string: 
            x = string.split(":")
            for string in x:
                line_string = '%s\n' % (string.strip())
                candidate_polymers_2.append(line_string)
        if "/" in string: 
            y = string.split("/")
            for string in y:
                line_string = '%s\n' % (string.strip())
                candidate_polymers_2.append(line_string)
        if "–" in string: 
            z = string.split("–")
            for string in z:
                line_string = '%s\n' % (string.strip())
                candidate_polymers_2.append(line_string)
        
        
    joined_list = ''.join(candidate_polymers_2)
    
    for line in joined_list: 
        if line not in candidate_polymers: 
            candidate_polymers.append(line)

    return candidate_polymers

In [6]:
def evaluate (candidate_file, groundtruth_file):
    
    candidate_polymers = read_items(candidate_file)
    extracted_polymers = read_items(groundtruth_file)
    
    print (len(candidate_polymers))
    print (len(extracted_polymers))

    groundtruth_polymers = [x.strip().lower() for x in extracted_polymers]
    groundtruth_polymers = list(set(groundtruth_polymers))
    print "Number of polymer names in the dictionary: ", len(groundtruth_polymers)

    candidate_polymers = [x.strip().lower() for x in candidate_polymers]
    candidate_polymers = list(set(candidate_polymers))
    print "Number of candidate polymers in this file: ", (len(candidate_polymers))
    
    # Perform evaluation (compute truth positives, false positives and false negatives)
    TP = 0
    FP = 0
    for candidate in candidate_polymers:
        if candidate in groundtruth_polymers:
            TP = TP + 1
        else:
            FP = FP + 1

    precision = (TP/(TP+FP)*100)

    FN = 0
    for poly in groundtruth_polymers:
        if poly not in candidate_polymers:
            FN = FN + 1

    recall = (TP/(TP+FN)*100)

    print "True pos: ", TP
    print "False pos: ", FP
    print "Precision: ", precision
    print "Recall: ", recall

In [7]:
#find partial matches between candidates and groundtruth

def partial_match (candidate_file, groundtruth_file):
    
    candidate_polymers = read_items(candidate_file)
    extracted_polymers = read_items(groundtruth_file)

    groundtruth_polymers = [x.strip().lower() for x in extracted_polymers]
    groundtruth_polymers = list(set(groundtruth_polymers))

    candidate_polymers = [x.strip().lower() for x in candidate_polymers]
    candidate_polymers = list(set(candidate_polymers))
    
    for candidate in groundtruth_polymers: 
        for item in candidate_polymers:
            if len(item) > 4: 
                if len(item) < len(candidate):
                    if item in candidate:
                        print(item)
                        print(candidate)
                    else: 
                        pass
                else: 
                    pass
            else: 
                pass

In [8]:
evaluate('output/amber_polymer_candidates.txt', 'evaluation/ground_truth_polymers_list.txt')